In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from threading import Thread
from time import sleep, time
from urllib.request import urlopen, Request

# Informações Importantes

In [2]:
tmdb = "https://www.themoviedb.org"

In [3]:
h = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

# Classe Item

In [4]:
class Item:
    
    def __init__(self, url=None, tipo='filme', id=None, name=None, html=None, download=True):
        self.tipo = tipo
        if url != None and download:
            self.url = url
            aux = url.replace(tmdb, '')
            self.id = int(aux.split('/')[-1]) if tipo == 'filme' else int(aux.split('/')[2].split('-')[0])

            while True:
                r = requests.get(url, headers=h)
                if r.status_code == 200:
                    break
                else:
                    sleep(1)
            self.html = bs(r.text, 'html.parser')
            self.name = self.html.find('div', {'class': 'title'}).find('a').text if tipo == 'filme' else self.html.find('h2', {'class': 'title'}).text
        else:
            self.id = id
            self.name = name
            self.html = html
            self.url = url
    
    def pegaId(self):
        return self.link.split('/')[-2] if self.tipo == 'filme' else self.link.split('/')[2].split('-')[0]
    
    def __str__(self):
        return f'{self.id} - {self.name} - {self.url} - {self.tipo} - html = {"TEM" if self.html != None else "Nao tem"}'

## Classe Filme

O filme deve possuir uma lista com seus principais atores, que virá no HTML.

In [5]:
class Filme(Item):
    def __init__(self, url):
        super().__init__(url=url, tipo='filme')
        aux = [(ator.find('a')['href'].split('/')[-1].split('-')[0], ator.find('p').text) 
               for ator in self.html.find_all('ol', {'class': 'people scroller'})[0].find_all('li')][:-1]
        self.elenco = [
            Item(
                id=int(x[0]),
                name=x[1], 
                tipo='ator', 
                url=f'{tmdb}/person/{x[0]}', 
                download=False) for x in aux
            ]

## Classe Ator

Um ator deve possuir uma lista com seus principais filmes, que virá no HTML.

In [6]:
class Ator(Item):
    def __init__(self, url):
        super().__init__(url=url, tipo='ator')
        ids = [ (
                    int(filme.find('a')['href'].split('/')[-1]),
                    filme.find('p').text
                ) for filme in self.html.find_all('div', id='known_for')[0].find_all('li')]
        self.know = [ 
            Item(
                id=i[0],
                name=i[1], 
                tipo='filme', 
                url=f"{tmdb}/movie/{i[0]}", 
                download=False) for i in ids
            ]

# Dois Atores que quero conectar

In [7]:
urlS1 = "https://www.themoviedb.org/person/234352-margot-robbie"
urlS2 = "https://www.themoviedb.org/person/23532-jason-bateman"

In [8]:
s1, s2 = Ator(url=urlS1), Ator(url=urlS2)

In [9]:
print(s1); print(s2)

234352 - Margot Robbie - https://www.themoviedb.org/person/234352-margot-robbie - ator - html = TEM
23532 - Jason Bateman - https://www.themoviedb.org/person/23532-jason-bateman - ator - html = TEM


In [10]:
for i in s1.know:
    print(i)

495764 - Birds of Prey (and the Fantabulous Emancipation of One Harley Quinn) - https://www.themoviedb.org/movie/495764 - filme - html = Nao tem
436969 - The Suicide Squad - https://www.themoviedb.org/movie/436969 - filme - html = Nao tem
106646 - The Wolf of Wall Street - https://www.themoviedb.org/movie/106646 - filme - html = Nao tem
297761 - Suicide Squad - https://www.themoviedb.org/movie/297761 - filme - html = Nao tem
389015 - I, Tonya - https://www.themoviedb.org/movie/389015 - filme - html = Nao tem
466272 - Once Upon a Time… in Hollywood - https://www.themoviedb.org/movie/466272 - filme - html = Nao tem
256591 - Focus - https://www.themoviedb.org/movie/256591 - filme - html = Nao tem
122906 - About Time - https://www.themoviedb.org/movie/122906 - filme - html = Nao tem


In [11]:
filme_teste = "https://www.themoviedb.org/movie/843847"

In [12]:
f  = Filme(url=filme_teste)

In [13]:
for x in f.elenco:
    print(x)

17419 - Bryan Cranston - https://www.themoviedb.org/person/17419 - ator - html = Nao tem
516 - Annette Bening - https://www.themoviedb.org/person/516 - ator - html = Nao tem
11678 - Rainn Wilson - https://www.themoviedb.org/person/11678 - ator - html = Nao tem
154091 - Larry Wilmore - https://www.themoviedb.org/person/154091 - ator - html = Nao tem
21731 - Michael McKean - https://www.themoviedb.org/person/21731 - ator - html = Nao tem
196181 - Ann Harada - https://www.themoviedb.org/person/196181 - ator - html = Nao tem
94791 - Jake McDorman - https://www.themoviedb.org/person/94791 - ator - html = Nao tem
221098 - Anna Camp - https://www.themoviedb.org/person/221098 - ator - html = Nao tem
2551620 - Devyn McDowell - https://www.themoviedb.org/person/2551620 - ator - html = Nao tem


# Inicio

In [14]:
mensagem = lambda filme, ator: f"O ator {ator.name} participou do filme {filme.name}"

In [15]:
def busca(item, lista):
    r = [x for x in lista if x.id == item.id]
    return len(r) > 0, r[0] if len(r) > 0 else None

In [16]:
def escolheAtor(atual:Filme, historico:list):
    escolhido = None
    i = 0
    while escolhido == None and i < len(atual.elenco):
        if busca(item=atual.elenco[i], lista=historico)[0] == False:
            escolhido = atual.elenco[i]
        i += 1
    return escolhido

In [17]:
def escolheFilme(atorAtual:Ator, historico:list):
    escolhido = None
    i = 0
    while escolhido == None and i < len(atorAtual.know):
        if busca(item=atorAtual.know[i], lista=historico)[0] == False:
            escolhido = atorAtual.know[i]
        i += 1
    return escolhido

In [18]:
def buscaSeq(movInit:Item, s2:Ator, historico:list, thr:int, encontrou=False, numMax=50, possivel=True):
    # Possivel = True: Ainda é possivel encontrar o caminho.
    # Possivel = False: Não é mais possivel encontrar o caminho pelos atores principais.
    # Acrenscentar uma trava a cada 35 requisições, já que o site só permite 40 por segundo.
    while len(historico) < numMax and encontrou == False and possivel:
        historico.append(Filme(url=movInit.url)) # Requisitar o filme e já colocar no historico.
        print(f"T{thr} | FILME: {historico[-1].name}")

        encontrou, ator = busca(item=s2, lista=historico[-1].elenco)

        # Encontrou?
        if encontrou: # Sim:
            print(f"T{thr} | {mensagem(filme=historico[-1], ator=ator)}")
            historico.append(s2)
        else: # Não:

            # Escolher o ator entre o elenco, deve ser alguém que não esteja no historico.
            atorAtual = escolheAtor(atual=historico[-1], historico=historico)

            if atorAtual != None: # Ainda é possivel encontrar o caminho.
                historico.append(Ator(url=atorAtual.url))
                print(f"T{thr} | ATOR: {historico[-1].name}")

                # Escolher o filme entre os filmes que o ator participou, deve ser um filme que não esteja no historico.
                movInit = escolheFilme(atorAtual=historico[-1], historico=historico)

            else:
                possivel = False



In [19]:
def conecta(s1, s2, qntThrs=1):
    numMax = 50
    encontrou = False
    historicos = [[s1] for _ in range(qntThrs)]
    limite = 35

    thrs = [Thread(target=buscaSeq, args=(s1.know[i], s2, historicos[i], i, encontrou, numMax)) for i in range(qntThrs)]
    
    print(thrs)

    for thr in thrs:
        thr.start()

    for thr in thrs:
        thr.join()

    return historicos


In [20]:
historicos = conecta(s1, s2, qntThrs=5)

[<Thread(Thread-7, initial)>, <Thread(Thread-8, initial)>, <Thread(Thread-9, initial)>, <Thread(Thread-10, initial)>, <Thread(Thread-11, initial)>]


T0 | FILME: Birds of Prey (and the Fantabulous Emancipation of One Harley Quinn)
T4 | FILME: I, Tonya
T2 | FILME: The Wolf of Wall Street
T1 | FILME: The Suicide Squad
T3 | FILME: Suicide Squad
T2 | ATOR: Leonardo DiCaprio
T4 | ATOR: Sebastian Stan
T0 | ATOR: Ewan McGregor
T1 | ATOR: Idris Elba
T3 | ATOR: Will Smith
T2 | FILME: Inception
T4 | FILME: Captain America: The Winter Soldier
T0 | FILME: Trainspotting
T1 | FILME: Zootopia
T1 | O ator Jason Bateman participou do filme Zootopia
T2 | ATOR: Joseph Gordon-Levitt
T3 | FILME: I Am Legend
T4 | ATOR: Chris Evans
T0 | ATOR: Ewen Bremner
T3 | ATOR: Alice Braga
T2 | FILME: Looper
T4 | FILME: Captain America: Civil War
T0 | FILME: Wonder Woman
T2 | ATOR: Bruce Willis
T3 | FILME: Soul
T4 | ATOR: Robert Downey Jr.
T2 | FILME: The Sixth Sense
T0 | ATOR: Gal Gadot
T4 | FILME: The Avengers
T2 | ATOR: Haley Joel Osment
T3 | ATOR: Jamie Foxx
T0 | FILME: Wonder Woman 1984
T4 | ATOR: Mark Ruffalo
T2 | FILME: A.I. Artificial Intelligence
T3 | FILME:

In [22]:
for i in historicos[0]:
    print(i)

234352 - Margot Robbie - https://www.themoviedb.org/person/234352-margot-robbie - ator - html = TEM
495764 - Birds of Prey (and the Fantabulous Emancipation of One Harley Quinn) - https://www.themoviedb.org/movie/495764 - filme - html = TEM
3061 - Ewan McGregor - https://www.themoviedb.org/person/3061 - ator - html = TEM
627 - Trainspotting - https://www.themoviedb.org/movie/627 - filme - html = TEM
1125 - Ewen Bremner - https://www.themoviedb.org/person/1125 - ator - html = TEM
297762 - Wonder Woman - https://www.themoviedb.org/movie/297762 - filme - html = TEM
90633 - Gal Gadot - https://www.themoviedb.org/person/90633 - ator - html = TEM
464052 - Wonder Woman 1984 - https://www.themoviedb.org/movie/464052 - filme - html = TEM
62064 - Chris Pine - https://www.themoviedb.org/person/62064 - ator - html = TEM
13475 - Star Trek - https://www.themoviedb.org/movie/13475 - filme - html = TEM
17306 - Zachary Quinto - https://www.themoviedb.org/person/17306 - ator - html = TEM
54138 - Star Tr